In [1]:
#%load_ext autoreload
#%autoreload 2

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('../modules'))

In [3]:
from bokeh.io import output_notebook, output_file, save, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot, column, row
from bokeh.models import ColumnDataSource, Legend, NumeralTickFormatter, Text
from bokeh.models import HoverTool,PanTool,ResetTool,HoverTool,WheelZoomTool,SaveTool,BoxZoomTool,UndoTool,RedoTool,SaveTool,ZoomInTool,ZoomOutTool
from bokeh.palettes import magma
import datetime

In [4]:
import own.funny as funny

import own.data.ecdc as ECDC
import own.data.countries as Countries

from own.colors import focus_countries

In [5]:
population = ECDC.population()
population = population.div(1000000)

population = population[population.index.isin(focus_countries.keys())]
population['population_rounded'] = population['population'].round(0)

population = population.merge(
    Countries.country_names()['common_name'][focus_countries],
    left_index=True, right_index=True
)

In [6]:
def TOOLS():
    return[
        HoverTool(),
        BoxZoomTool(dimensions='both'),
        PanTool(dimensions='both'),
        ZoomInTool(),
        ZoomOutTool(),
        ResetTool(),
        SaveTool(),
        UndoTool(),
        RedoTool(),
    ]

In [7]:
#population = population.sort_values(by='population', ascending=False) #.iloc[0:25]

source = ColumnDataSource(data=dict(
    country_code = population.index.values.tolist(),
    country_name = population['common_name'],
    population = population['population'],
    population_rounded = population['population_rounded'],
))


p = figure(
    plot_height = 250, plot_width = 583,
#    x_range = population.index.values.tolist(),
    x_range = population['common_name'],
    y_axis_label = 'Population size (mio)',
    y_axis_location = 'right',
    tools = TOOLS(),
    title='Population size (mio)',
    
    tooltips = [
        ("Country", "@country_name"),
        ("Population (mio)", "@population"),
    ],
)

p.yaxis[0].formatter = NumeralTickFormatter(format="1,000")

p.xaxis.major_label_orientation = 0
p.yaxis.minor_tick_line_color = None

p.xaxis.major_tick_line_color = None
p.xaxis.major_label_text_font_size = '9.5pt'
p.xaxis.major_label_text_font_style = 'bold'
p.xaxis.axis_line_width = 0
p.xaxis.axis_line_color = None

p.y_range.start = 0

p.xgrid.visible = False

p.ygrid.grid_line_color = 'blue'
p.ygrid.grid_line_alpha = 0.1


p.background_fill_color = '#f0f0f0'
p.background_fill_alpha = 0.20

p.toolbar.logo = None

p.x_range.range_padding = 0.025

p.vbar(
    x='country_name', 
    top='population',
    bottom=0.001,
    source=source,
    width=0.67,
    muted_alpha=0.33,
)

from bokeh.models import LabelSet

labels = LabelSet(
    x='country_name', y='population',
    text='population_rounded',
#    text='country_name',
    level='glyph',
    x_offset=-15.5, y_offset=2,
    source=source,
    render_mode='css', #render_mode='canvas',
    text_color='#000066',
    text_alpha=0.67,
    text_font_size='9pt',
    border_line_color='gray',
    border_line_alpha=0.1,
    background_fill_color='#ffffff',
    background_fill_alpha=0.5,
)
p.add_layout(labels)

grid = gridplot(
    [p],
    ncols = 1,
    sizing_mode='scale_both',
    merge_tools = True,
    toolbar_options = dict(logo=None),
    toolbar_location = 'right',
)

if funny.is_save_for_later():
    funny.save_for_later('ecdc-bar', grid)

if funny.is_save_to_public():
    output_file('../docs/plots/ecdc-bar.html', mode='inline')#, mode='relative-dev', root_dir='../lib/bokeh/2.0.1')
    save(grid)

if funny.is_display():
    output_notebook()
    show(grid)